<a href="https://colab.research.google.com/github/scienstien/A-repositry/blob/main/basic_task_SPIDER0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

import os
import kagglehub

#Download latest version
path = kagglehub.dataset_download("mohamedmaher5/vehicle-classification")

print("Path to dataset files:", path)
print("Files in dataset:")
print(os.listdir(path))

Path to dataset files: /kaggle/input/vehicle-classification
Files in dataset:
['Vehicles']


# IMPORTS


In [3]:
import os
import torch
import torchvision
import torchvision.transforms as transforms

# Prepare Dataset


In [4]:
dataset_dir =  '/kaggle/input/vehicle-classification/Vehicles'

# for root, dirs, files in os.walk(dataset_dir):
#   print(f"{root} has {len(files)} files.")

# TRANSFORMS

In [5]:
transforms = transforms.Compose([
    transforms.Resize(226),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
     transforms.Normalize(mean = [0.485, 0.456, 0.406],
                          std = [0.229, 0.224, 0.225])

])

# DATA LOADER


In [8]:
from torchvision import datasets, models
from torch.utils.data import DataLoader

dataset = datasets.ImageFolder(dataset_dir, transform=transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size,val_size])



train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [9]:
dataset.classes
dataset.class_to_idx


{'Auto Rickshaws': 0,
 'Bikes': 1,
 'Cars': 2,
 'Motorcycles': 3,
 'Planes': 4,
 'Ships': 5,
 'Trains': 6}

# RESNET MODEL

In [10]:
import torch
import torch.nn as nn


if torch.cuda.is_available():
  device = torch.device('cuda')
else :
  device = torch.device('cpu')

resnet_model = models.resnet18(pretrained = True)

num_classes = len(dataset.classes)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)

model = resnet_model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 182MB/s]


In [ ]:
resnet_model.state_dict()

#LOSS AND OPTIMISERS FUNCTION

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training  LOOP

In [ ]:
epochs =  0
for epoch in range(epochs) :
  resnet_model.train()
  for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
  #forward pass
  predicted_label = resnet_model(images)
  # Calculate Losses
  loss = loss_fn(predicted_label, labels)
  #Zero GRAd
  optimiser.zero_grad()
  #Back Propogtion
  loss.backward()
  # step function
  optimiser.step()



# Saving the model

In [11]:
from pathlib import Path
MODEL_PATH = Path('models')
MODEL_PATH.mkdir(parents = True , exist_ok = True)

MODEL_NAME = "basic_task_SPIDER.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME
print(os.listdir(torch.save(resnet_model.state_dict(), MODEL_SAVE_PATH)))


['.config', 'drive', 'models', 'sample_data']
